# Import

In [59]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Dense, SimpleRNN, Flatten
from keras import Model, Sequential, layers, regularizers, optimizers
from keras.callbacks import EarlyStopping
from overcome_tomorrow.utils.data import *
from overcome_tomorrow.utils.preprocess import *
pd.set_option("display.max_columns", None)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load model

In [2]:
X = merge_all_data("../raw_data/Wellness/", "../raw_data/Fitness/", "../raw_data/Aggregator/")
y = pd.read_csv("../raw_data/activities.csv")

✅ ../raw_data/Wellness/ loaded
✅ Wellness cleaned
✅ ../raw_data/Fitness/ loaded
✅ Fitness cleaned


/home/ruben/code/Ruuuben06/overcome-tomorrow/overcome-tomorrow/overcome_tomorrow/utils/data.py:180: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_fitness_df["date"] = pd.to_datetime(clean_fitness_df["beginTimestamp"], unit="ms").dt.date
/home/ruben/code/Ruuuben06/overcome-tomorrow/overcome-tomorrow/overcome_tomorrow/utils/data.py:181: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_fitness_df["date"] = pd.to_datetime(clean_fitness_df["date"])
/home/ruben/code/Ruuuben06/overcome-tomorrow/overcome-t

✅ ../raw_data/Aggregator/ loaded
✅ Aggregator cleaned
✅ Datasets merged


In [3]:
X.head()

,start_sleep,end_sleep,deepSleepSeconds,lightSleepSeconds,remSleepSeconds,awakeSleepSeconds,qualityScore,beginTimestamp,activityTrainingLoad,activityType,...,minAvgHeartRate,maxAvgHeartRate,hydration_sweatLossInML,respiration_avgWakingRespirationValue,respiration_highestRespirationValue,respiration_lowestRespirationValue,bodyBattery_Highest,bodyBattery_Lowest,allDayStress_awake,allDayStress_asleep
calendarDate,,,,,,,,,,,,,,,,,,,,,
2019-08-29,2019-08-28 20:30:00,2019-08-29 05:00:00,NaN,NaN,NaN,NaN,NaN,2019-08-29 19:23:03,5.244644,walking,...,50.0,101.0,NaN,NaN,NaN,NaN,69.0,45.0,18.0,11.0
2019-08-29,2019-08-28 20:30:00,2019-08-29 05:00:00,NaN,NaN,NaN,NaN,NaN,2019-08-29 17:41:54,2.278595,other,...,50.0,101.0,NaN,NaN,NaN,NaN,69.0,45.0,18.0,11.0
2019-08-29,2019-08-28 20:30:00,2019-08-29 05:00:00,NaN,NaN,NaN,NaN,NaN,2019-08-29 09:39:26,NaN,running,...,50.0,101.0,NaN,NaN,NaN,NaN,69.0,45.0,18.0,11.0
2019-08-30,2019-08-29 21:12:00,2019-08-30 05:08:00,5340.0,17940.0,4920.0,360.0,NaN,2019-08-30 17:38:42,8.112671,strength_training,...,46.0,119.0,NaN,NaN,NaN,NaN,100.0,66.0,20.0,6.0
2019-08-30,2019-08-29 21:12:00,2019-08-30 05:08:00,5340.0,17940.0,4920.0,360.0,NaN,2019-08-30 17:19:21,1.880997,other,...,46.0,119.0,NaN,NaN,NaN,NaN,100.0,66.0,20.0,6.0


In [4]:
y.head()

,timestamp,total_anaerobic_training_effect,enhanced_avg_speed,avg_stroke_distance,total_training_effect,num_active_lengths,max_running_cadence,max_heart_rate,sub_sport,total_descent,...,normalized_power,training_load_peak,total_ascent,avg_power,total_strokes,training_stress_score,avg_cadence,avg_heart_rate,max_power,activity_id
0,2024-02-25 16:48:13+00:00,0.0,1.234,NaN,0.2,NaN,NaN,111.0,generic,1.0,...,NaN,3.143585,19.0,NaN,NaN,NaN,52.0,96.0,NaN,nicko64@hotmail.fr_237829993545
1,2022-11-08 07:39:34+00:00,0.0,7.023,NaN,0.6,NaN,NaN,132.0,gravel_cycling,20.0,...,165.0,9.253098,20.0,144.0,880.0,8.7,82.0,110.0,466.0,nicko64@hotmail.fr_161107489254
2,2020-01-16 12:18:52+00:00,0.3,4.197,NaN,4.6,NaN,95.0,179.0,generic,52.0,...,NaN,257.602173,62.0,NaN,NaN,NaN,92.0,166.0,NaN,nicko64@hotmail.fr_47505547526
3,2023-11-25 07:52:52+00:00,0.0,4.108,NaN,3.3,NaN,97.0,167.0,generic,34.0,...,357.0,118.865219,35.0,354.0,NaN,NaN,88.0,147.0,516.0,nicko64@hotmail.fr_221724422608
4,2021-11-24 19:47:49+00:00,0.0,1.468,NaN,0.1,NaN,NaN,92.0,generic,13.0,...,NaN,1.332031,25.0,NaN,NaN,NaN,54.0,70.0,NaN,nicko64@hotmail.fr_114450316357


In [61]:
y.sort_index().head()

,timestamp,total_anaerobic_training_effect,enhanced_avg_speed,avg_stroke_distance,total_training_effect,num_active_lengths,max_running_cadence,max_heart_rate,sub_sport,total_descent,start_time,total_distance,pool_length,178,total_calories,max_cadence,sport,188,avg_step_length,enhanced_max_speed,205,num_lengths,206,207,pool_length_unit,normalized_power,training_load_peak,total_ascent,avg_power,total_strokes,training_stress_score,avg_cadence,avg_heart_rate,max_power,activity_id
calendarDate,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-08-29,2019-08-29 19:49:41+00:00,0.0,1.387,NaN,0.5,NaN,NaN,96.0,generic,32.0,2019-08-29 19:23:03+00:00,2192.8,NaN,NaN,121.0,59.0,walking,0.0,NaN,1.987,NaN,NaN,NaN,NaN,NaN,NaN,5.244644,31.0,NaN,NaN,NaN,52.0,81.0,NaN,nicko64@hotmail.fr_39756984532
2019-08-29,2019-08-29 17:52:57+00:00,0.0,0.000,NaN,0.2,NaN,NaN,114.0,generic,0.0,2019-08-29 17:41:54+00:00,0.0,NaN,NaN,24.0,65.0,generic,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.278595,0.0,NaN,NaN,NaN,10.0,89.0,NaN,nicko64@hotmail.fr_39751851901
2019-08-30,2019-08-30 17:58:50+00:00,0.0,0.000,NaN,0.8,NaN,NaN,131.0,strength_training,NaN,2019-08-30 17:38:42+00:00,0.0,NaN,NaN,103.0,NaN,training,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.112671,NaN,NaN,NaN,NaN,NaN,107.0,NaN,nicko64@hotmail.fr_39806439039
2019-08-30,2019-08-30 17:30:17+00:00,0.0,0.000,NaN,0.1,NaN,NaN,106.0,generic,0.0,2019-08-30 17:19:21+00:00,0.0,NaN,NaN,24.0,66.0,generic,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.880997,0.0,NaN,NaN,NaN,6.0,89.0,NaN,nicko64@hotmail.fr_39805289042
2019-08-31,2019-08-31 11:19:57+00:00,2.0,0.833,1.6,3.2,49.0,NaN,167.0,lap_swimming,NaN,2019-08-31 10:30:48+00:00,2450.0,50.0,NaN,611.0,NaN,swimming,2.0,NaN,3.636,NaN,49.0,NaN,NaN,metric,NaN,100.325455,NaN,NaN,1533.0,NaN,31.0,139.0,NaN,nicko64@hotmail.fr_39844611872


#  Preprocessing

In [5]:
X_preprocessed = preproc_garmin_data(X)

✅ Preprocess successful


In [11]:
X_preprocessed.shape

(3445, 66)

In [8]:
X_preprocessed.head()

,knn_transformer__deepSleepSeconds,knn_transformer__lightSleepSeconds,knn_transformer__remSleepSeconds,knn_transformer__awakeSleepSeconds,knn_transformer__qualityScore,knn_transformer__activityTrainingLoad,knn_transformer__aerobicTrainingEffect,knn_transformer__anaerobicTrainingEffect,knn_transformer__avgBikeCadence,knn_transformer__avgHr,...,pipe_categorical__trainingEffectLabel_SPEED,pipe_categorical__trainingEffectLabel_TEMPO,pipe_categorical__trainingEffectLabel_UNKNOWN,pipe_categorical__trainingEffectLabel_VO2MAX,cycle transform__start_sleep_sin,cycle transform__start_sleep_cos,cycle transform__end_sleep_sin,cycle transform__end_sleep_cos,cycle transform__beginTimestamp_sin,cycle transform__beginTimestamp_cos
calendarDate,,,,,,,,,,,,,,,,,,,,,
2019-08-29,0.132812,-0.152344,0.700980,0.020833,-0.400,-0.344406,-0.571429,0.0,-2.340426,-0.857143,...,0.0,0.0,1.0,0.0,-0.866025,0.500000,0.965926,0.258819,-0.965926,0.258819
2019-08-29,0.031250,0.371094,0.235294,0.041667,-0.400,-0.375987,-0.678571,0.0,-0.638298,-0.693878,...,0.0,0.0,1.0,0.0,-0.866025,0.500000,0.965926,0.258819,-0.965926,-0.258819
2019-08-29,0.132812,-0.152344,0.700980,0.020833,-0.400,0.638791,0.348214,0.0,-0.170213,0.632632,...,0.0,0.0,1.0,0.0,-0.866025,0.500000,0.965926,0.258819,0.707107,-0.707107
2019-08-30,0.781250,0.531250,-0.333333,0.083333,-0.975,-0.313870,-0.464286,0.0,-2.297872,-0.326531,...,0.0,0.0,1.0,0.0,-0.707107,0.707107,0.965926,0.258819,-0.965926,-0.258819
2019-08-30,0.781250,0.531250,-0.333333,0.083333,-0.975,-0.380220,-0.714286,0.0,-0.638298,-0.693878,...,0.0,0.0,1.0,0.0,-0.707107,0.707107,0.965926,0.258819,-0.965926,-0.258819


In [7]:
y_preprocessed = preproc_activity(y)

✅ Preprocess successful


In [10]:
y_preprocessed.shape

(3363, 31)

In [9]:
y_preprocessed.head()

,knn_imputer__total_anaerobic_training_effect,knn_imputer__enhanced_avg_speed,knn_imputer__178,knn_imputer__max_heart_rate,knn_imputer__enhanced_max_speed,knn_imputer__206,knn_imputer__188,knn_imputer__207,knn_imputer__total_training_effect,knn_imputer__total_descent,...,cat_encoder__sport_rock_climbing,cat_encoder__sport_running,cat_encoder__sport_swimming,cat_encoder__sport_training,cat_encoder__sport_transition,cat_encoder__sport_walking,cycle_encoder__timestamp_sin,cycle_encoder__timestamp_cos,cycle_encoder__start_time_sin,cycle_encoder__start_time_cos
0,0.0,-0.085266,-0.567518,-0.704545,-0.210953,0.892086,-1.0,0.863014,-0.714286,-0.329545,...,0.0,0.0,0.0,0.0,0.0,1.0,-8.660254e-01,-0.500000,-0.866025,-0.500000
1,0.0,0.876922,-0.751825,-0.227273,0.894001,1.208633,-1.0,1.164384,-0.571429,-0.113636,...,0.0,0.0,0.0,0.0,0.0,0.0,9.659258e-01,-0.258819,0.965926,-0.258819
2,0.3,0.407213,0.468978,0.840909,0.124428,-0.273381,1.0,-0.356164,0.857143,0.250000,...,0.0,1.0,0.0,0.0,0.0,0.0,1.224647e-16,-1.000000,0.258819,-0.965926
3,0.0,0.392421,0.383212,0.568182,0.145210,0.000000,0.0,0.013699,0.392857,0.045455,...,0.0,1.0,0.0,0.0,0.0,0.0,9.659258e-01,-0.258819,0.965926,-0.258819
4,0.0,-0.046372,-0.414234,-1.136364,-0.229251,0.633094,-1.0,0.602740,-0.750000,-0.193182,...,0.0,0.0,0.0,0.0,0.0,1.0,-9.659258e-01,0.258819,-0.965926,0.258819


In [35]:
y_preprocessed.columns

Index(['knn_imputer__total_anaerobic_training_effect',
       'knn_imputer__enhanced_avg_speed', 'knn_imputer__178',
       'knn_imputer__max_heart_rate', 'knn_imputer__enhanced_max_speed',
       'knn_imputer__206', 'knn_imputer__188', 'knn_imputer__207',
       'knn_imputer__total_training_effect', 'knn_imputer__total_descent',
       'knn_imputer__avg_heart_rate', 'knn_imputer__total_calories',
       'knn_imputer__training_load_peak', 'knn_imputer__total_distance',
       'knn_imputer__total_ascent', 'imputer 100__205',
       'cat_encoder__sport_cycling', 'cat_encoder__sport_fitness_equipment',
       'cat_encoder__sport_generic', 'cat_encoder__sport_hiking',
       'cat_encoder__sport_kayaking', 'cat_encoder__sport_rock_climbing',
       'cat_encoder__sport_running', 'cat_encoder__sport_swimming',
       'cat_encoder__sport_training', 'cat_encoder__sport_transition',
       'cat_encoder__sport_walking', 'cycle_encoder__timestamp_sin',
       'cycle_encoder__timestamp_cos', 'cycle

In [36]:
X_preprocessed.columns

Index(['knn_transformer__deepSleepSeconds',
       'knn_transformer__lightSleepSeconds',
       'knn_transformer__remSleepSeconds',
       'knn_transformer__awakeSleepSeconds', 'knn_transformer__qualityScore',
       'knn_transformer__activityTrainingLoad',
       'knn_transformer__aerobicTrainingEffect',
       'knn_transformer__anaerobicTrainingEffect',
       'knn_transformer__avgBikeCadence', 'knn_transformer__avgHr',
       'knn_transformer__avgPower', 'knn_transformer__avgRunCadence',
       'knn_transformer__avgSpeed', 'knn_transformer__calories',
       'knn_transformer__caloriesConsumed', 'knn_transformer__distance',
       'knn_transformer__duration', 'knn_transformer__maxHr',
       'knn_transformer__maxPower', 'knn_transformer__maxRunCadence',
       'knn_transformer__maxSpeed',
       'knn_transformer__moderateIntensityMinutes',
       'knn_transformer__normPower', 'knn_transformer__trainingStressScore',
       'knn_transformer__vigorousIntensityMinutes',
       'knn_trans

# Create model

In [67]:
def initialize_model(input_shape):
    model = Sequential()
    model.add(layers.LSTM(64, input_shape=input_shape))
    model.add(layers.Dense(input_shape[1], activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model